# Análise de Currículos Lattes — Relatório

## Título e Integrantes

Projeto: Análise de Currículos Lattes (CNPq)

Equipe:

- Sarah Fernanda Calixto de Araújo — `20240011267`
- Luiz Carlos Veloso de Araujo Lima Neto — `20240102334`
- Sofia Pontes Leitão de Lima — `20240011285`
- Cauã Henrique Formiga de Lacerda — `20240011089`


## Introdução

Este relatório apresenta a análise exploratória e modelagem de dados extraídos da Plataforma Lattes, com foco em compreender padrões de produção acadêmica, redes de colaboração e temas de pesquisa de docentes/pesquisadores. O trabalho está estruturado para documentar objetivos, dados, pré-processamento, perguntas analíticas e conclusões.

Objetivos específicos:

- Mapear o perfil dos pesquisadores a partir de seus currículos (resumo, formação, produções, colaborações).
- Explorar a distribuição e evolução temporal da produção acadêmica.
- Identificar estruturas de colaboração (coautorias, projetos) e possíveis comunidades.
- Investigar fatores associados à produtividade científica.
- Descobrir agrupamentos temáticos de pesquisadores via técnicas de clustering.

Contexto técnico do projeto:

- Coleta com Playwright (navegação e captura dos HTMLs dos currículos).
- Extração com BeautifulSoup e consolidação com Pandas.
- Scripts principais em `scripts/` e artefatos em `data/` e `professores_perfil_html/`.


## Dados usados (descrição e exploração inicial)

Arquivos e fontes do projeto:

| Arquivo/Path                       | Papel                                       | Observações                                    |
| ---------------------------------- | ------------------------------------------- | ---------------------------------------------- |
| `data/professores_ci.csv`          | Lista de entrada com docentes/pesquisadores | Ponto de partida para scraping.                |
| `professores_perfil_html/`         | HTMLs dos currículos Lattes                 | Saída do script de download com Playwright.    |
| `data/professores.csv`             | Dados consolidados (CSV)                    | Saída do parser (`scripts/parse_profiles.py`). |
| `data/professores.json`            | Dados consolidados (JSON)                   | Alternativa de saída do parser.                |
| `data/prof_labs.csv`               | Mapeamento professor ↔ laboratório          | Usado para análises por laboratório/grupo.     |
| `data/publicacoes_unificadas.json` | Produções agregadas (quando disponível)     | Facilita análises temporais e por tipo.        |

Perguntas de exploração inicial (EDA):

- Quantos perfis foram coletados e processados? Há registros ausentes (NAs) em campos-chave?
- Como é a distribuição de produção por pesquisador, por ano e por tipo (artigo, conferência, capítulo, etc.)?
- Quais palavras-chave/termos são mais frequentes em resumos e títulos de publicações?
- Como se distribuem os pesquisadores por laboratório/grupo (quando houver `prof_labs.csv`)?

Sugestões de visualização:

- Histogramas/boxplots da produção por pesquisador e por ano.
- Séries temporais de publicações agregadas por ano.
- Nuvens de palavras ou barplots de termos mais frequentes.


## Pré-processamento (limpeza e transformações)

Passos principais aplicados aos dados consolidados:

1. Padronização de textos: normalização de acentuação, lowercasing seletivo (mantendo siglas), remoção de quebras e espaços extras.
2. Parsing de datas: conversão de campos de ano/mês para tipos temporais, imputação conservadora quando faltantes (ex.: usar apenas o ano quando o mês não existir).
3. Tratamento de valores ausentes: estratégias específicas por campo (ex.: preencher `resumo` vazio com placeholder controlado; remover linhas só quando estritamente necessário).
4. Deduplicação: remoção de duplicatas de publicações e coautores usando chaves compostas (título normalizado + ano + veículo).
5. Enriquecimento: criação de variáveis derivadas (ex.: contagem de publicações por tipo/ano; tempo desde o doutorado; tamanho da rede de coautoria; indicadores por laboratório).
6. Tokenização e limpeza para texto livre (resumos e títulos): remoção de stopwords, lematização/stemming quando aplicável e construção de representações (TF‑IDF, embeddings) para análises temáticas.

Decisões e justificativas:

- Priorizou-se manter registros com dados parciais (minimizando viés de remoção) e documentar imputações.
- Normalização forte de chaves textuais para reduzir duplicatas por variações menores (case/acentos).
- Features de texto foram mantidas em duas granulometrias: termos frequentes (interpretação) e vetores densos (modelos).


## Perguntas (lista resumida)

1- Quais são os professores com mais contribuições e coautores?
2- Qual é a relação entre a quantidade de anos de experiência e a produção bibliográfica?
3- Como o nível de formação influencia a produção bibliográfica (mestrado, doutorado e pós-doutorado)?
4- Como é o grafo de produção com coautoria?
5- Quais clusters emergem a partir dos títulos dos artigos?.


### 1- Quais são os professores com mais contribuições e coautores?

- Métricas: total de publicações por pesquisador; número distinto de coautores; taxa de coautoria (coautores/publicações).
- Abordagem: agregar `publicacoes` por pesquisador e extrair coautores únicos a partir de cada produção; criar ranking e top‑N.
- Visualização: barplots ordenados (top‑N) e heatmap de coautorias entre top pesquisadores.
- Observações: tratar homônimos (normalização de nomes) e possíveis duplicatas em coautorias.


### 2- Qual é a relação entre a quantidade de anos de experiência e a produção bibliográfica?

- Definição: anos de experiência = ano de referência − ano da conclusão do doutorado (ou mestrado, na ausência de doutorado).
- Modelo: regressão linear simples/múltipla da produção anual em função dos anos de experiência e controles (lab, área, tamanho da rede).
- Avaliação: RMSE/MAE, resíduos e análise de efeitos marginais; checar não linearidade (polinômios ou splines, se necessário).
- Visualização: scatter com linha de tendência e intervalos de confiança; facetas por laboratório/área.


### 3- Como o nível de formação influencia a produção bibliográfica (mestrado, doutorado e pós-doutorado)?

- Variáveis: dummies/níveis categóricos de formação máxima e indicadores de pós‑doutorado.
- Abordagem: modelos lineares (ANOVA/OLS) comparando médias condicionais de produção; controles por experiência e rede.
- Interpretação: diferenças médias associadas a cada nível; intervalos de confiança; tamanho de efeito.
- Visualização: boxplots/violin por nível de formação, com sobreposição de médias/IC.


### 4- Como é o grafo de produção com coautoria?

- Construção: grafo não direcionado com nós = pesquisadores e arestas ponderadas pelo número de publicações em coautoria.
- Métricas: grau, betweenness, eigenvector; comunidades (Louvain/Leiden).
- Visualização: layout forçado (Fruchterman‑Reingold/ForceAtlas2) com cor por comunidade e tamanho por centralidade.
- Saídas: top nós por centralidade e subgrafos das comunidades principais.


### 5- Quais clusters emergem a partir dos títulos dos artigos?.

- Representação: TF‑IDF de títulos (ou embeddings) agregados por pesquisador ou por publicação.
- Algoritmos: K‑Means (baseline) e/ou HDBSCAN; redução UMAP para 2D.
- Validação: Silhouette/Davies‑Bouldin; inspeção de termos dominantes por cluster.
- Visualização: nuvem de palavras por cluster e scatter 2D (UMAP) colorido por cluster.


## Conclusão

Síntese esperada:

- Ranking de pesquisadores por produção e coautoria e a estrutura de colaboração observada.
- Relações entre experiência, nível de formação e produtividade.
- Perfis temáticos identificados a partir dos títulos dos artigos.

Limitações:

- Lacunas e desatualizações nos currículos; ambiguidade de nomes/identificadores.
- Diferenças de cobertura entre áreas e anos; variações na classificação de tipos de produção.

Trabalhos futuros e melhorias:

- Desambiguação com ORCID/DOI e enriquecimento via bases externas.
- Modelos de NLP mais robustos (embeddings contextuais) e análise temporal fina de tópicos.
- Dashboards interativos para acompanhamento contínuo.

> Observação: Este notebook se baseia nos scripts do repositório (`scripts/download_profile.py` e `scripts/parse_profiles.py`). Para reproduzir a coleta e parsing, siga as instruções do `README.md` (uso de `uv`).
